# Canadian house price analysis
## Sections

## Motivation
- Recent inflation number and house price records have constantly make news. This report aims to replicate [the analysis carried out by Bank of Canada in 2015](https://www.bankofcanada.ca/2015/08/long-term-evolution-house-prices/) using more up-to-date data from Statistics Canada.

## Data importation and cleaning

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os

### Importing data

In [6]:
pwd = os.getcwd()
#Load main data
df = pd.read_csv(pwd+'\\data\\raw\\housing\\18100205.csv',header = 0)
#Get table of metadata
df_md = pd.read_csv(pwd+'\\data\\raw\\housing\\18100205_MetaData.csv',header = 0, skiprows =7,nrows = 43)

### Data cleaning 

In [7]:
#Change REF_DATE into yyyy-mm date type
df['Month'] = pd.to_datetime(df.REF_DATE)
#Get data for all Canada
df_Can = df[(df.GEO == "Canada") ]
#Condition to get total houe value: df["New housing price indexes"].str.contains('Total')
#Get provinces name from metadata file
unique_province_id = df_md['Parent Member ID'].unique()
provinces_names = df_md[(df_md['Member ID'].isin(unique_province_id)) & (df_md['Dimension ID']==1) \
    & ~(df_md['Member ID'].isin([1,2,28]))]['Member Name']
#Get a dictionary province_df to store each province data where keys are name of provinces, values are dictionary
province_df = {}
for name in provinces_names:
    province_df[name] = df[df.GEO == name]

## Data exploration

### Overview of Canada and the provinces

ModuleNotFoundError: No module named 'geopandas'